In [6]:
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection, naive_bayes, svm, tree
from sklearn.metrics import accuracy_score
from sklearn.linear_model import RidgeClassifier
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
import gzip
import json
import string

In [7]:
np.random.seed(500)

In [8]:
df = pd.read_csv(r"C:\Users\stezo\Desktop\Text_classification\Software.csv")
df.head()

,Unnamed: 0,rating,verified,reviewTime,reviewerID,productID,reviewText,summary,unixReviewTime
0,0,4,True,"03 11, 2014",A240ORQ2LF9LUI,0077613252,The materials arrived early and were in excell...,Material Great,1394496000
1,1,4,True,"02 23, 2014",A1YCCU0YRLS0FE,0077613252,I am really enjoying this book with the worksh...,Health,1393113600
2,2,1,True,"02 17, 2014",A1BJHRQDYVAY2J,0077613252,"IF YOU ARE TAKING THIS CLASS DON""T WASTE YOUR ...",ARE YOU KIDING ME?,1392595200
3,3,3,True,"02 17, 2014",APRDVZ6QBIQXT,0077613252,This book was missing pages!!! Important pages...,missing pages!!,1392595200
4,4,5,False,"10 14, 2013",A2JZTTBSLS1QXV,0077775473,I have used LearnSmart and can officially say ...,Best study product out there!,1381708800


In [9]:
df['label'] = df['rating'].apply(lambda x: 1 if x > 3 else 0 )
amazon_data = df.loc[:,('reviewText','label')]
amazon_data.columns = ['text', 'label']

amazon_data

,text,label
0,The materials arrived early and were in excell...,1
1,I am really enjoying this book with the worksh...,1
2,"IF YOU ARE TAKING THIS CLASS DON""T WASTE YOUR ...",0
3,This book was missing pages!!! Important pages...,0
4,I have used LearnSmart and can officially say ...,1
...,...,...
423348,I love it .,1
423349,No instructions.....No Help unless you want to...,0
423350,it's a joke,0
423351,I have multiple licenses of the Antivirus. I h...,1


In [10]:
def clean_text(text ): 
    delete_dict = {sp_character: '' for sp_character in string.punctuation} 
    delete_dict[' '] = ' ' 
    table = str.maketrans(delete_dict)
    text1 = str(text).translate(table)
    #print('cleaned:'+text1)
    textArr= text1.split()
    text2 = ' '.join([w for w in textArr if ( not w.isdigit() and  ( not w.isdigit() and len(w)>2))]) 
    
    return text2.lower()

In [11]:
amazon_data['text'].dropna(axis = 0, how ='any',inplace=True) 

amazon_data['text'] = amazon_data['text'].apply(clean_text)

In [12]:
amazon_data[amazon_data['label']==0]

,text,label
2,you are taking this class dont waste your mone...,0
3,this book was missing pages important pages co...,0
6,you live mars and never heard the internet thi...,0
10,maybe its just have marketing background but d...,0
12,not worth the price most the concepts are pret...,0
...,...,...
423345,the images are deceptive the pendant small you...,0
423346,small its hard make out what when you wear,0
423347,fault shorter than expected,0
423349,instructionsno help unless you want pay for help,0


In [13]:
# Podział na zbiór trenningowy oraz testowy.
Train_X, Test_X, Train_Y, Test_Y = model_selection.train_test_split(amazon_data['text'],
                                                                    amazon_data['label'],
                                                                    train_size=0.4,
                                                                    test_size=0.3)

# Kodowanie etykiety
Encoder = LabelEncoder()
Train_Y = Encoder.fit_transform(Train_Y)
Test_Y = Encoder.fit_transform(Test_Y)

# Wektoryzacja tekstu
Tfidf_vect = TfidfVectorizer(max_features=5000)
Tfidf_vect.fit(amazon_data['text'])
Train_X_Tfidf = Tfidf_vect.transform(Train_X)
Test_X_Tfidf = Tfidf_vect.transform(Test_X)

In [14]:
print(list(Tfidf_vect.vocabulary_.items())[:5])

[('the', 4439), ('materials', 2667), ('arrived', 279), ('early', 1341), ('and', 200)]


In [15]:
print("Size of the training set: ", Train_X_Tfidf.shape[0])
print("Size of the training set: ", Test_X_Tfidf.shape[0])

Size of the training set:  169341
Size of the training set:  127006


In [61]:
# Klasyfikator liniowy
Linear = RidgeClassifier(alpha=0.05, 
                         fit_intercept=True, 
                         normalize='deprecated', 
                         copy_X=True, 
                         max_iter=None, 
                         tol=0.001, 
                         class_weight=None, 
                         solver='auto', 
                         random_state=None)

%time Linear.fit(Train_X_Tfidf,Train_Y)

predictions_Ridge = Linear.predict(Test_X_Tfidf)

print("Ridge Classifier Accuracy Score -> ",accuracy_score(predictions_Ridge, Test_Y))

Wall time: 242 ms
Ridge Classifier Accuracy Score ->  0.8639355620994283


In [62]:
# Niwny klasyfikator Bayesowski
Naive = naive_bayes.MultinomialNB(alpha=0.05, 
                                  fit_prior=True, 
                                  class_prior=None)

%time Naive.fit(Train_X_Tfidf,Train_Y)

predictions_NB = Naive.predict(Test_X_Tfidf)

print("Naive Bayes Accuracy Score -> ",accuracy_score(predictions_NB, Test_Y))

Wall time: 25.7 ms
Naive Bayes Accuracy Score ->  0.8398185912476576


In [16]:
# Maszyna wektorów nośnych
SVM = svm.SVC(C=0.5, 
              kernel='linear', 
              degree=3, 
              gamma='auto', 
              coef0=0.0, 
              shrinking=True, 
              probability=False, 
              tol=0.001, 
              cache_size=200, 
              class_weight=None, 
              verbose=False, 
              max_iter=- 1, 
              decision_function_shape='ovr', 
              break_ties=False, 
              random_state=None)

%time SVM.fit(Train_X_Tfidf,Train_Y)

predictions_SVM = SVM.predict(Test_X_Tfidf)

print("SVM Accuracy Score -> ",accuracy_score(predictions_SVM, Test_Y)*100)

Wall time: 1h 7min 38s
SVM Accuracy Score ->  87.33445663984378


In [11]:
# Drzewa decyzyjne
Tree = tree.DecisionTreeClassifier(criterion='entropy', 
                                   splitter='best',
                                   max_features='auto',
                                   max_depth=None, 
                                   min_samples_split=2, 
                                   min_samples_leaf=1, 
                                   min_weight_fraction_leaf=0.0,  
                                   random_state=None, 
                                   max_leaf_nodes=None, 
                                   min_impurity_decrease=0.0, 
                                   class_weight=None, 
                                   ccp_alpha=0.0)

%time Tree.fit(Train_X_Tfidf,Train_Y)

predictions_Tree = Tree.predict(Test_X_Tfidf)

print("Decision Tree Accuracy Score -> ",accuracy_score(predictions_Tree, Test_Y))

Wall time: 5.99 s
Decision Tree Accuracy Score ->  0.727800261404973


In [12]:
# Losowy Las
Forest = RandomForestClassifier(n_estimators=100,
                                criterion='entropy',
                                max_features='auto',
                                n_jobs=-1,
                                random_state=None,
                                verbose=1)

%time Forest.fit(Train_X_Tfidf,Train_Y)

predictions_Forest = Forest.predict(Test_X_Tfidf)

print("Random Forest Accuracy Score -> ",accuracy_score(predictions_Forest, Test_Y))

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:   34.6s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:  1.7min finished
[Parallel(n_jobs=12)]: Using backend ThreadingBackend with 12 concurrent workers.


Wall time: 1min 42s


[Parallel(n_jobs=12)]: Done  26 tasks      | elapsed:    0.2s


Random Forest Accuracy Score ->  0.8460230225343685


[Parallel(n_jobs=12)]: Done 100 out of 100 | elapsed:    0.8s finished
